In [3]:
d = {
    'batch_size': 777,
    'n_devices': 4,
    'gradient_accumulation_steps': 8,
}

current_batch_size_per_gpu = d["batch_size"] // (
    d["gradient_accumulation_steps"] * d["n_devices"]
)  # This value assures that the num_batch_chunks is 1

num_batch_chunks = max(
    d["gradient_accumulation_steps"]
    // (d["batch_size"] // (current_batch_size_per_gpu * d["n_devices"])),
    1,
)

print(f"current_batch_size_per_gpu: {current_batch_size_per_gpu}")
print(f"num_batch_chunks: {num_batch_chunks}")

current_batch_size_per_gpu: 24
num_batch_chunks: 1


In [11]:
def convert_transition_points_in_tokens_to_steps(
    transition_points_in_tokens: list[float], batch_sizes: list[int], seq_len: int
):
    transition_points_in_steps = []
    transition_points_in_tokens = [p * 1e9 for p in transition_points_in_tokens]
    tokens_per_step_list = [batch_size * seq_len for batch_size in batch_sizes]
    steps_prev = tokens_prev = 0

    for point, tokens_per_step in zip(
        transition_points_in_tokens, tokens_per_step_list
    ):
        tokens_to_transition = point - tokens_prev
        steps_to_transition = tokens_to_transition / tokens_per_step
        point_in_steps = steps_prev + steps_to_transition

        transition_points_in_steps.append(int(point_in_steps))

        tokens_prev = point
        steps_prev = point_in_steps
    return transition_points_in_steps

def convert_tokens_to_steps(
    tokens: int,  # in bilions
    seq_len: int,
    target_batch_size: int,
    transition_points: list[int] = None,
    batch_sizes: list[int] = None,
):
    if transition_points is None:
        return int((tokens) / (target_batch_size * seq_len))
    
    tokens

    tokens_per_step_list = [batch_size * seq_len for batch_size in batch_sizes]
    steps_prev = tokens_prev = 0

    for point, tokens_per_step in zip(transition_points, tokens_per_step_list):
        steps_to_transition = point - steps_prev
        tokens_to_trasition = steps_to_transition * tokens_per_step
        tokens_current = tokens_prev + tokens_to_trasition

        print(f'tokens_current: {tokens_current}')

        if tokens < tokens_current:
            return int(steps_prev + (tokens - tokens_prev) / tokens_per_step)

        tokens_prev = tokens_current
        steps_prev = point

    # After all ramp-up intervals
    return int(steps_prev + (tokens - tokens_prev) / (target_batch_size * seq_len))

tokens = 0.262144
target_batch_size = 512
seq_len = 512
tp = [0.01, 0.05]
bsz = [128, 256]

tp = convert_transition_points_in_tokens_to_steps(
    transition_points_in_tokens=tp,
    batch_sizes=bsz,
    seq_len=seq_len
)

print(f'tp: {tp}')

steps = convert_tokens_to_steps(
    tokens=tokens * 1e9,
    seq_len=seq_len,
    target_batch_size=target_batch_size,
    transition_points=tp,
    batch_sizes=bsz,
)

print(steps)

tp: [152, 457]
tokens_current: 9961472
tokens_current: 49938432
1266


queisoqu9Hag